In [ ]:
## Correct intersection

In [2]:
import source_estimator
from pathlib import Path

In [6]:
pip install rtree

  Obtaining dependency information for rtree from https://files.pythonhosted.org/packages/1c/a6/524326a993fad04215299e0023ed9c93b15d5bd62822d887729a90c27d52/Rtree-1.1.0-py3-none-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.2/398.2 kB 5.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
subjects_dir = '/Applications/freesurfer/7.4.1/subjects/inspected'

In [7]:
source_estimator.correct_intersecting_meshes(Path(subjects_dir), 'sub-032508', suffix="")

inner:  /Applications/freesurfer/7.4.1/subjects/inspected/sub-032508/surf/lh.pial
outer:  /Applications/freesurfer/7.4.1/subjects/inspected/sub-032508/bem/inner_skull.surf
The inner surface intersect the outer surface. Pushing back the outer surface 0.5 mm out of the inner surface. Saving the outer surface as /Applications/freesurfer/7.4.1/subjects/inspected/sub-032508/bem/inner_skull.surf.
inner:  /Applications/freesurfer/7.4.1/subjects/inspected/sub-032508/surf/rh.pial
outer:  /Applications/freesurfer/7.4.1/subjects/inspected/sub-032508/bem/inner_skull.surf
The inner surface intersect the outer surface. Pushing back the outer surface 0.5 mm out of the inner surface. Saving the outer surface as /Applications/freesurfer/7.4.1/subjects/inspected/sub-032508/bem/inner_skull.surf.
inner:  /Applications/freesurfer/7.4.1/subjects/inspected/sub-032508/bem/inner_skull.surf
outer:  /Applications/freesurfer/7.4.1/subjects/inspected/sub-032508/bem/outer_skull.surf
The inner surface intersect the 

{'lh.pial': array([0., 0., 0., ..., 0., 0., 0.]),
 'rh.pial': array([0., 0., 0., ..., 0., 0., 0.]),
 'inner_skull.surf': array([0., 0., 0., ..., 0., 0., 0.]),
 'outer_skull.surf': array([0., 0., 0., ..., 0., 0., 0.])}

In [ ]:
## Calculating coh

In [4]:
import mne
from mne.coreg import Coregistration
from mne.minimum_norm import make_inverse_operator, apply_inverse_epochs
import numpy as np
from pathlib import Path
import pandas as pd
import os
from pathlib import Path
from mne_connectivity import spectral_connectivity_epochs
import xarray as xr
import time

In [5]:
def filter_labels_with_vertices(labels_parc, src):
    # Get the vertices from both hemispheres in the source space
    src_vertices = [src[0]['vertno'], src[1]['vertno']]
    
    # Initialize an empty list to hold valid labels
    valid_labels = []
    
    for label in labels_parc:
        # Determine the hemisphere index: 0 for 'lh' and 1 for 'rh'
        hemi_idx = 0 if label.hemi == 'lh' else 1
        
        # Check if any of the label's vertices are in the source space for that hemisphere
        if any(v in src_vertices[hemi_idx] for v in label.vertices):
            valid_labels.append(label)
            
    return valid_labels


In [8]:
subject = 'sub-032475'
conditions = ['EC','EO']

In [10]:
conditions_mats = []
for condition in conditions:
    src = mne.setup_source_space(subject, add_dist="patch", subjects_dir=subjects_dir)
    raw = mne.io.read_raw_eeglab(f"/Users/lina_01/Downloads/eeg/{subject}/{subject}_{condition}.set")
    
    info = raw.info
    fiducials = "estimated"
    coreg = Coregistration(info, subject, subjects_dir, fiducials=fiducials)
    
    conductivity = (0.3, 0.006, 0.3)
    model = mne.make_bem_model(subject=subject, conductivity=conductivity, subjects_dir=subjects_dir)
    bem = mne.make_bem_solution(model)
    
    epochs = mne.make_fixed_length_epochs(raw, duration=1.0, preload=False)
    epochs.set_eeg_reference(projection=True)
    epochs.apply_baseline((None,None))
    fwd = mne.make_forward_solution(
        epochs.info, trans=coreg.trans, src=src, bem=bem, verbose=True
    )
    
    cov = mne.compute_covariance(epochs)
    
    inv = mne.minimum_norm.make_inverse_operator(epochs.info, fwd, cov, verbose=True)
    
    method = "sLORETA"
    snr = 3.0
    lambda2 = 1.0 / snr**2
    stc = apply_inverse_epochs(
        epochs,
        inv,
        lambda2,
        method=method,
        pick_ori=None,
        verbose=True,
        return_generator=False
    )
    
    labels_parc = mne.read_labels_from_annot(subject, parc='aparc', subjects_dir=subjects_dir)
    
    filtered_labels = filter_labels_with_vertices(labels_parc, src)
    label_ts = mne.extract_label_time_course(stc, filtered_labels, src, mode='auto', return_generator=False, allow_empty=False)
    
    n=len(epochs)

    coh_mats=[]
    for i in range(100):
        inds = np.random.choice(range(n),int(n/2),replace=False)
        mne_con = spectral_connectivity_epochs(np.array(label_ts)[inds], 
                                           method="coh", sfreq=250, mode='multitaper', 
                                           fmin=8, fmax=13, fskip=0, faverage=False,
                                           tmin=None, tmax=None, mt_bandwidth=None, mt_adaptive=False,
                                           mt_low_bias=True, block_size=1000, n_jobs=1, verbose=None)
        mat = np.real(mne_con.get_data(output="dense"))
        coh_mats.append(mat)

    region = [label.name for label in filtered_labels]
    frequencies = list(mne_con.freqs)
    bootstrap_samples = list(range(100))

    xarray = xr.DataArray(np.array(coh_mats), dims=["bootstrap_samples","region1","region2","frequencies"],
    coords={"bootstrap_samples":bootstrap_samples,"region1":region, "region2":region,"frequencies":frequencies})
    xarray.to_netcdf(f'{subject}_array_coh_{condition}.nc')

Setting up the source space with the following parameters:

SUBJECTS_DIR = /Applications/freesurfer/7.4.1/subjects
Subject      = sub-032475
Surface      = white
Octahedron subdivision grade 6

>>> 1. Creating the source space...

Doing the octahedral vertex picking...
Loading /Applications/freesurfer/7.4.1/subjects/sub-032475/surf/lh.white...
Mapping lh sub-032475 -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from /Applications/freesurfer/7.4.1/subjects/sub-032475/surf/lh.sphere...
Setting up the triangulation for the decimated surface...
loaded lh.white 4098/148881 selected to source space (oct = 6)

Loading /Applications/freesurfer/7.4.1/subjects/sub-032475/surf/rh.white...
Mapping rh sub-032475 -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from /Applications/freesurfer/7.4.1/subjects/sub-032475/surf/rh.sphere...
Setting up the triangulation for the decimated surface...
loaded rh.white 4098/149162 selected to source space

/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_848/1740981872.py:4: RuntimeWarning: Data file name in EEG.data (sub-010268_EC.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-032475_EC.fdt).
  raw = mne.io.read_raw_eeglab(f"/Users/lina_01/Downloads/eeg/{subject}/{subject}_{condition}.set")
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_848/1740981872.py:4: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(f"/Users/lina_01/Downloads/eeg/{subject}/{subject}_{condition}.set")
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_848/1740981872.py:4: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(f"/Users/lina_01/Downloads/eeg/{subject}/{subject}_{condition}.set")


Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
outer skin  CM is   0.22 -21.49  11.67 mm
outer skull CM is   0.24 -21.33  11.34 mm
inner skull CM is  -0.25 -26.51  19.13 mm
Checking that surface outer skull is inside surface outer skin  ...


RuntimeError: Surface outer skull is not completely inside surface outer skin 